In [1]:
import numpy as np
import tvm
from tvm import te

In [2]:
def matmul(n, m, l):
    """Return the computing expression of matrix multiplication
    A : n x l matrix
    B : l x m matrix
    C : n x m matrix with C = A B
    """
    k = te.reduce_axis((0, l), name='k')
    A = te.placeholder((n, l), name='A')
    B = te.placeholder((l, m), name='B')
    C = te.compute((n, m),
                    lambda x, y: te.sum(A[x, k] * B[k, y], axis=k),
                    name='C')
    return A, B, C

In [3]:
n = 100
A, B, C = matmul(n, n, n)
s = te.create_schedule(C.op)
print(tvm.lower(s, [A, B], simple_mode=True))
mod = tvm.build(s, [A, B, C])

@main = primfn(A_1: handle, B_1: handle) -> ()
  attr = {"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True}
  buffers = {A: Buffer(A_2: Pointer(float32), float32, [10000], []),
             B: Buffer(B_2: Pointer(float32), float32, [10000], [])}
  buffer_map = {A_1: A, B_1: B}
  preflattened_buffer_map = {A_1: A_3: Buffer(A_2, float32, [100, 100], []), B_1: B_3: Buffer(B_2, float32, [100, 100], [])} {
  allocate(C: Pointer(global float32), float32, [10000]), storage_scope = global;
  for (x: int32, 0, 100) {
    for (y: int32, 0, 100) {
      C_1: Buffer(C, float32, [10000], [])[((x*100) + y)] = 0f32
      for (k: int32, 0, 100) {
        let cse_var_2: int32 = (x*100)
        let cse_var_1: int32 = (cse_var_2 + y)
        C_1[cse_var_1] = (C_1[cse_var_1] + (A[(cse_var_2 + k)]*B[((k*100) + y)]))
      }
    }
  }
}




In [5]:
def get_abc(shape, constructor=None):
    """Return random a, b and empty c with the same shape.
    """
    np.random.seed(0)
    a = np.random.normal(size=shape).astype(np.float32)
    b = np.random.normal(size=shape).astype(np.float32)
    c = np.empty_like(a)
    if constructor:
        a, b, c = [constructor(x) for x in (a, b, c)]
    return a, b, c

In [6]:
a, b, c = get_abc((100, 100), tvm.nd.array)
mod(a, b, c)
np.testing.assert_allclose(np.dot(a.asnumpy(), b.asnumpy()),
                           c.asnumpy(), atol=1e-5)